In [45]:
class Variable:
    def __init__(self, data):
        self.data = data

In [46]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [47]:
class Function:
    def __call__(self, input):
        x = input.data
        y = x**2
        output = Variable(y)
        return output

In [48]:
x = Variable(np.array(10))
f = Function()
y= f(x)

print(type(y))
print(y.data)


<class '__main__.Variable'>
100


In [49]:
# __call__  매서드는 클래스를 함수처럼 사용가능   f = Fuction() 형태로 인스턴스를 생성하고 나서  f(input) 형태로 사용가능


In [50]:
class Function:
    def __call__(self, input):
        x= input.data
        y = self.forward(x)
        output = Variable(y)
        return output
    
    def forward(self, x):
        raise NotImplementedError()

In [51]:
f = Function()
f.forward(x)

NotImplementedError: 

In [52]:
# Fuction 클래스의 forward 메서드는 예외를 발생시킵니다. 이렇게 해두면 Function  클래스의 forward  메서드를 직접 호출한 사람에게
# 이 메서드는 상속해서 구현해야 한다는것을 알려줄 수 있습니다.


In [53]:
class Square(Function):
    def forward(self, x):
        return x**2

In [54]:
x = Variable(np.array(10))
f = Square()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


# 함수 연결

In [55]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    

In [68]:
# 함수 합성
A = Square()
B = Exp()
C = Square()

x = Variable(np.array((0.5)))
a = A(x)
b = B(a)
y = C(b)

print(y.data)

1.648721270700128


# 수치미분

In [69]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2*eps)

In [70]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [72]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))
x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


# 역전파

In [73]:
class Variable:
    def __init__(self, data):
        self.data  = data
        self.grad = None # 역전파시 미분값을 계산하여 대입 합니다.
        

In [75]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input # 입력 변수를 기억(보관)한다.
        return output
    
    
    def forward(self, x):
        raise NotImplementedError
        
    def backward(self, gy):
        raise NotImplementedError
        
        
        
        

In [76]:
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2*x*gy
        return gx
    
    

In [77]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [80]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
      
      

In [82]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256
